In [25]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
import ast
import pandas as pd
from urllib.request import urlretrieve
import urllib
import json

def get_states(url):
    states = None
    driver = webdriver.Chrome(executable_path="./chromedriver")
    driver.get(url)
    ## We have to wait till the element has loaded
    try:
        elem = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "state")) #This is a dummy element
        )
        selector = Select(elem)
        options = selector.options
        states = [option.text for option in options]
    finally:
        driver.quit()    
    
    return states


def get_L1_data(state):
    new_url = "https://facilities.aicte-india.org/dashboard/pages/faculties.php?method=fetchdata&state="+state+"&program=ENGINEERING AND TECHNOLOGY&level=1&institutiontype=1&Women=1&Minority=1"
    
    state_url = new_url.replace(" ","%20")
    #print(state_url)    
    response = requests.get(state_url)
    print(response.text)
    return response.json()

def consolidateL2ResponseToDataFrame(dictionary, explodeColumns):
    data_f = pd.DataFrame([dictionary]).T
    data_f.columns = ['all_data']
    
    #Removing rows with no data Orissa
    data_f = data_f[~data_f.all_data.isnull()]
    data_f = data_f.explode('all_data')     
    data_f[explodeColumns] = pd.DataFrame(data_f.all_data.tolist(), index= data_f.index)
    data_f = data_f.drop(columns = ['all_data'])
    return data_f  
    

In [27]:
base_url= "https://facilities.aicte-india.org/dashboard/pages/faculties.php"
states = get_states(base_url)
print(states)
states = states[:1]+states[2:]
print(states)
state_data = {state: get_L1_data(state) for state in states}
print(state_data)
state_data = {}
for state in tqdm(states):
    if state not in state_data:
        state_data[state] = get_L1_data(state)

print(f'Total {len(state_data)} states information fetched')

/tmp/ipykernel_692990/473295534.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="./chromedriver")


['Andaman and Nicobar Islands', '-------------------------------------------', 'Arunachal Pradesh', 'Andhra Pradesh', 'Andaman and Nicobar Islands', 'Assam', 'Bihar', 'Chandigarh', 'Chhattisgarh', 'Dadra and Nagar Haveli', 'Daman and Diu', 'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Orissa', 'Puducherry', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal']
['Andaman and Nicobar Islands', 'Arunachal Pradesh', 'Andhra Pradesh', 'Andaman and Nicobar Islands', 'Assam', 'Bihar', 'Chandigarh', 'Chhattisgarh', 'Dadra and Nagar Haveli', 'Daman and Diu', 'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Orissa', 'Pudu

JSONDecodeError: [Errno Expecting value]  
<!DOCTYPE html>
<html>

<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">

    <meta http-equiv="X-UA-Compatible" content="IE=edge">
   
    <meta name="description" content="">
    <meta name="author" content="">
<link rel="icon" href="images/aictetitle.ico" type="image/x-icon" />
	  <link rel="shortcut icon" href="../../images/aictetitle.ico" type="image/x-icon" />
      <title>AICTE</title>
  
    <!-- MetisMenu CSS -->
    <link href="../bower_components/metisMenu/dist/metisMenu.min.css" rel="stylesheet">
<link rel="icon" href="images/aictetitle.ico" type="image/x-icon" />
	 
	  <link rel="stylesheet" href="https://cdn.datatables.net/1.10.13/css/jquery.dataTables.min.css">
	<link rel="stylesheet" href="dist/ladda.min.css">
          <!-- Custom Fonts -->
    <link href="../bower_components/font-awesome/css/font-awesome.min.css" rel="stylesheet" type="text/css">
  
<style>
	

In [3]:
exploded_columns = ['aicteId','Name','Address','Distict','InstitutionType','Women','Minority','secondaryId']
states_df = consolidateL2ResponseToDataFrame(state_data, exploded_columns)
states_df.to_csv('states.csv')

In [4]:
len(state_data['Assam'])

49

In [5]:
details_page = 'https://facilities.aicte-india.org/dashboard/pages/php/approvedcourse.php?method=fetchdata&aicteid=/{}/&course=/1/&year=/2021-2022/'

In [ ]:
import time
i=0

details = {}
for aicteId in tqdm(states_df.aicteId.tolist()):    
    # try:
    response = requests.get(details_page.format(aicteId))
    time.sleep(1)
    i += 1
    # print(f'{i}/8988',end="\r")
    #print(details_page.format(aicteId))
    if aicteId not in details:
        details[aicteId] = response.json()
    else:
        print(f'Duplicate for {aicteId}')    
    # except:
    #     # print("Connection refused by the server..")
    #     # print("Let me sleep for 5 seconds")
    #     # print("ZZzzzz...")
    #     time.sleep(5)
    #     # print("Was a nice sleep, now let me continue...")
    #     continue


In [8]:
exploded_columns_details = ['aicteId','Name','State','Programme','University','LevelOfCourse','Course','Shift', 'Type' ,'Intake','Enrollment','Placement']
details_df = consolidateL2ResponseToDataFrame(details, exploded_columns_details)
details_df.to_csv('course_details1.csv',index=False)

In [10]:
faculty_page = 'https://facilities.aicte-india.org/dashboard/pages/php/faculty.php?method=fetchdata&aicteid={0}&pid={1}&year=2020-2021'

In [12]:
faculty = {}
i=0
for index,row in tqdm(states_df[['aicteId','secondaryId']].iterrows()):
    try:
        response = requests.get(faculty_page.format(row['aicteId'],row['secondaryId']))
        # time.sleep(1)
        i += 1
        # print(f'{i}/8988',end="\r")
        #print(details_page.format(aicteId))
        if aicteId not in faculty:
            if len(response.content) > 0 :
                faculty[row['aicteId']+'_'+row['secondaryId']] = response.json()           
        else:
            print(f'Duplicate for {aicteId}') 
    except:
        time.sleep(5)
        continue

5926it [1:34:44,  1.04it/s]


In [13]:
exploded_columns_faculty = ['facultyId','Name','Gender','Post','DateofHire','Speciality','AppointmentType','Institute Name']
faculty_df = consolidateL2ResponseToDataFrame(faculty, exploded_columns_faculty)
faculty_df.to_csv('faculty_details.csv')